In [ ]:
import pandas as pd
import numpy as np
import pickle

from tqdm.notebook import tqdm

from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
sns.set_style('darkgrid')

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso

from scipy.integrate import simps
from scipy.stats import norm
from scipy.stats import rankdata
from scipy.stats import kendalltau
from scipy.stats import spearmanr

from datetime import datetime

import random
import math

from xgboost import XGBRegressor
import multiprocessing
from multiprocessing.pool import Pool

import array
import random
import sys

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp


from sklearn.preprocessing import MinMaxScaler
import time


# !pip install kneed 
from kneed import KneeLocator

In [ ]:
def load_pickle(file):
    try:
        pickle_in= open(file, "rb")
        data = pickle.load(pickle_in)
        pickle_in.close()
        return data
    except IOError:
        print("Error loading pickle data from " + file)
        
    return None

def write_pickle(data, file):    
    try:
        pickle_out = open(file, "wb")
        pickle.dump(data, pickle_out, protocol=pickle.HIGHEST_PROTOCOL)
        pickle_out.close()
        return True
    except IOError:
        print("Error writing data to " + file)
        
    return False


#genetic distance utility functions
# Define fitness function and individuals structure
creator.create("fitness", base.Fitness, weights=(-1.0,)) # minimize a single objective (-1) or maximize it (+1)


def my_sqrt(arg1):
    return np.nan_to_num(np.sqrt(arg1))
def my_log(arg1):
    return np.nan_to_num(np.log(arg1))
def my_abs(arg1):
    return np.nan_to_num(np.abs(arg1))
def my_neg(arg1):
    return np.nan_to_num(np.negative(arg1))
def my_square(arg1):
    return np.nan_to_num(np.square(arg1))
def my_add(arg1, arg2):
    return np.nan_to_num(np.add(arg1, arg2))
def my_sub(arg1, arg2):
    return np.nan_to_num(np.subtract(arg1, arg2))
def my_mul(arg1, arg2):
    return np.nan_to_num(np.multiply(arg1, arg2))
def my_div(arg1, arg2):   
    return np.nan_to_num(np.divide(arg1, arg2))
def my_pow(arg1, arg2):
    return np.nan_to_num(np.power(float(arg1), float(arg2)))
def my_max(arg1, arg2):
    return np.nan_to_num(np.maximum(arg1, arg2), nan=sys.float_info.min)
def my_min(arg1, arg2):
    return np.nan_to_num(np.minimum(arg1, arg2), nan=sys.float_info.max)


pset = gp.PrimitiveSet("MAIN", arity=3)
pset.addPrimitive(my_add, 2)
pset.addPrimitive(my_sub, 2)
pset.addPrimitive(my_mul, 2)
pset.addPrimitive(my_div, 2)
pset.addPrimitive(my_pow, 2)
pset.addPrimitive(my_max, 2)
pset.addPrimitive(my_min, 2)
pset.addPrimitive(my_log, 1)
pset.addPrimitive(my_sqrt, 1)
pset.addPrimitive(my_square, 1)
pset.addPrimitive(my_abs, 1)
pset.addPrimitive(my_neg, 1)

pset.addEphemeralConstant("my_weight_1", lambda: random.uniform(-1, 1))
pset.addEphemeralConstant("my_weight_2", lambda: random.uniform(-1, 1))
pset.addEphemeralConstant("my_weight_3", lambda: random.uniform(-1, 1))
# pset.addEphemeralConstant("my_weight_4", lambda: random.uniform(-1, 1))
# pset.addEphemeralConstant("my_weight_5", lambda: random.uniform(-1, 1))


# Now create the Individual class
creator.create("Individual", gp.PrimitiveTree, fitness=creator.fitness, pset=pset)
toolbox = base.Toolbox() # this is a containr for all individuals, functions, operators, etc.

toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=5) # generator function to initialize the population individuals. "genHalfAndHalf" generates the trees in a list format, parameters: min_height, max_height
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr) # initializer for the elements of the population: a weight for each fingerprint distance function
toolbox.register("population", tools.initRepeat, list, toolbox.individual) # initializer for the population
toolbox.register("compile", gp.compile, pset=pset) # needed to compile the mathematical expressions

# Load best genetic programming function
# results = load_pickle('genetic_partial_results.pickle')
# gp_func=toolbox.compile(expr=list(results)[0][0])

In [ ]:
# Reload dataframe from disk, if it exists
data = load_pickle('extended_dataframe.pickle')

In [ ]:
# Now I delete all days containing NaN values
# Better to do proceed in this way, so not to have strange cut points

rows_with_nans = np.where(np.isnan(data['temperature'].values))[0]
dates_with_nans = np.unique(data['only_date'][rows_with_nans])
dates_indexes = data.index[data['only_date'].isin(dates_with_nans)].tolist()
print("Percentuale di dati scartati:", len(dates_indexes) / len(data))
data = data.drop(data.index[dates_indexes]).reset_index(drop=True)


data['year_week'] = data.datetime.dt.strftime('%Y-%U')

# For each sensor, I determine the indexes to partition its data into training and test
# In doing that, I keep into account the temporal relationships
# Meaning, test data is in the future wrt training data

perc_training = 0.8


split_type = 'week_based_random' # temporal_wise, day_based_random, full_random

    
# Checking that every sensor has the same length
counts_per_sensor = data[['node', 'datetime']].groupby('node').count()
assert np.max(counts_per_sensor['datetime']) == np.min(counts_per_sensor['datetime'])
sensor_data_length = counts_per_sensor['datetime'][0]    

   
# Maps that store the training and test indexes for each sensor
sens_map_train_indexes = {}
sens_map_test_indexes = {}

for sens in list(counts_per_sensor.index):
    sens_map_train_indexes[sens] = []
    sens_map_test_indexes[sens] = []    
    
    
    
if split_type == 'temporal_wise':
    # first 70% of each sensor data is traning, and next 30% is test

    # Generate training and test indexes

    # I have already sorted the values by node and datetime in the original dataframe
    for i, sens in enumerate(list(counts_per_sensor.index)):
        base_sens = i*sensor_data_length
        last_train = base_sens + int(sensor_data_length*perc_training)
        last_test = last_train + int(sensor_data_length*(1 - perc_training)) + 1
        train_inds = list(range(base_sens, last_train))
        test_inds = list(range(last_train, last_test))
        sens_map_train_indexes[sens] = train_inds
        sens_map_test_indexes[sens] = test_inds
        
    for sens in list(counts_per_sensor.index):
        first_train = sens_map_train_indexes[sens][0]
        last_train = sens_map_train_indexes[sens][-1]
        first_test = sens_map_test_indexes[sens][0]
        last_test = sens_map_test_indexes[sens][-1]
        assert first_train == data.query("node == @sens").reset_index().iloc[0,0]
        assert last_test == data.query("node == @sens").reset_index().iloc[-1,0]
        assert last_train > first_train and last_train < first_test
        assert first_test > last_train and first_test < last_test
    
elif split_type == 'full_random':
    # divide randomly the data of the sensors
    
    possible_indexes = list(range(sensor_data_length))
    random.seed(42)
    # I make sure to use the same split for each sensor
    # meaning, corresponding training and test indexes of the different sensors refer to the same datetimes
    train_indexes_overall = sorted(random.sample(possible_indexes, int(sensor_data_length * perc_training)))
    test_indexes_overall = sorted(list(set(possible_indexes) - set(train_indexes_overall)))
    
    for i, sens in enumerate(list(counts_per_sensor.index)):
        base_sens = i * sensor_data_length
        # it is sufficent to add the baseline, since all sensor tracks have the same length
        sens_map_train_indexes[sens] = np.asarray(train_indexes_overall) + base_sens
        sens_map_test_indexes[sens] = np.asarray(test_indexes_overall) + base_sens

    
    assert np.array_equal(sens_map_train_indexes['raspihat01'], sens_map_train_indexes['raspihat03'] - sensor_data_length*2)
    assert np.array_equal(sens_map_test_indexes['raspihat01'], sens_map_test_indexes['raspihat03'] - sensor_data_length*2)
    assert len(set(sens_map_train_indexes['raspihat01']).intersection(sens_map_test_indexes['raspihat01'])) == 0
    assert len(set(sens_map_train_indexes['raspihat01']).union(sens_map_test_indexes['raspihat01'])) == len(possible_indexes)
    # The training and test indexes are different for each sensor, and refer to the original dataframe

elif split_type == 'day_based_random':
    
    np.random.seed(42)
    days_list = data.only_date.unique().tolist()
    np.random.shuffle(days_list)
    random_train_days = sorted(days_list[0:int(len(days_list)*perc_training)])
    random_test_days =  sorted(days_list[int(len(days_list)*perc_training):])

    for i, sens in enumerate(list(counts_per_sensor.index)):
        subdata = data[data['node'] == sens] # here the indexes are still the same as in the original frame
        sens_map_train_indexes[sens] = subdata[subdata.only_date.isin(random_train_days)].index.tolist().copy()
        sens_map_test_indexes[sens] = subdata[subdata.only_date.isin(random_test_days)].index.tolist().copy()        
    
    assert np.array_equal(sens_map_train_indexes['raspihat01'], sens_map_train_indexes['raspihat03'] - sensor_data_length*2)
    assert np.array_equal(sens_map_test_indexes['raspihat01'], sens_map_test_indexes['raspihat03'] - sensor_data_length*2)
    assert len(set(sens_map_train_indexes['raspihat01']).intersection(sens_map_test_indexes['raspihat01'])) == 0
    assert len(set(sens_map_train_indexes['raspihat01']).union(sens_map_test_indexes['raspihat01'])) == len(data[data['node'] == 'raspihat01'])
    # The training and test indexes are different for each sensor, and refer to the original dataframe

elif split_type == 'week_based_random':
    
    np.random.seed(42)
    weeks_list = data.year_week.unique().tolist()
    np.random.shuffle(weeks_list)
    random_train_weeks = sorted(weeks_list[0:int(len(weeks_list)*perc_training)])
    random_test_weeks =  sorted(weeks_list[int(len(weeks_list)*perc_training):])

    for i, sens in enumerate(list(counts_per_sensor.index)):
        subdata = data[data['node'] == sens] # here the indexes are still the same as in the original frame
        sens_map_train_indexes[sens] = subdata[subdata.year_week.isin(random_train_weeks)].index.tolist().copy()
        sens_map_test_indexes[sens] = subdata[subdata.year_week.isin(random_test_weeks)].index.tolist().copy()        
    
    assert np.array_equal(sens_map_train_indexes['raspihat01'], sens_map_train_indexes['raspihat03'] - sensor_data_length*2)
    assert np.array_equal(sens_map_test_indexes['raspihat01'], sens_map_test_indexes['raspihat03'] - sensor_data_length*2)
    assert len(set(sens_map_train_indexes['raspihat01']).intersection(sens_map_test_indexes['raspihat01'])) == 0
    assert len(set(sens_map_train_indexes['raspihat01']).union(sens_map_test_indexes['raspihat01'])) == len(data[data['node'] == 'raspihat01'])
    # The training and test indexes are different for each sensor, and refer to the original dataframe
    
else:
    assert False, 'Unsupported split type'
    

In [ ]:
# Some general information

window_coords = [(1, 0), (3, 0), (6, 0), (8, 0), (10, 0),
                 (1, 9), (3, 9), (6, 9), (8, 9), (10, 9)]

all_train_idxs = []
for sens in sens_map_train_indexes:
    all_train_idxs.extend(sens_map_train_indexes[sens])

all_test_idxs = []
for sens in sens_map_test_indexes:
    all_test_idxs.extend(sens_map_test_indexes[sens])

node_map_coords = {}
for sens in sens_map_test_indexes:
    coord_x, coord_y = data[data['node'] == sens].iloc[0][['coord_x', 'coord_y']].values
    node_map_coords[sens] = (coord_x, coord_y)
    
all_sensors = np.unique(data['node'])

all_training_data = data.iloc[all_train_idxs].reset_index(drop=True)
all_test_data = data.iloc[all_test_idxs].reset_index(drop=True)

def eucl_dist(x_0, y_0, x_1, y_1):
    return np.sqrt((x_0 - x_1)**2 + (y_0 - y_1)**2)

training_data_ts = len(all_training_data[all_training_data['node'] == 'raspihat01'])

test_data_ts = len(all_test_data[all_test_data['node'] == 'raspihat01'])

air_tube_coords = [(x, 4) for x in list(range(11))]

def array_rep(arr, n):
    assert len(arr.shape) <= 2
    if len(arr.shape) < 2:
        return np.tile(arr, n)
    else:
        retarr = np.full((arr.shape[0]*n, arr.shape[1]), -1.)
        for i in range(n):
            retarr[i*arr.shape[0]: (i+1)*arr.shape[0]] = arr
        return retarr

In [ ]:
### Calcolo rank distanze con metriche base

# Now that we have a baseline for how to discard sensors using the temperature information,
# Let us see how closely we can match them using proxy information (e.g., spatial info)

# So, for each sensor, I now determine the rank of the other sensors


# determines the angle between two points (x_0, y_0) (x_1, y_1)
# if p2 is the origin (0, 0), then I get the angle wrt x axis
def angle_between(p1, p2):
    ang1 = np.arctan2(*p1[::-1])
    ang2 = np.arctan2(*p2[::-1])
    return np.rad2deg((ang1 - ang2) % (2 * np.pi))

def angle_between_center(p1, p2):
    ang1 = np.arctan2(*np.subtract(p1, (5,5))[::-1])
    ang2 = np.arctan2(*np.subtract(p2, (5,5))[::-1])
    return np.rad2deg((ang1 - ang2) % (2 * np.pi))

# Maps store the results from the best to the worst sensor, according to the specific metric
sensor_map_euclidean_ranks = {}
sensor_map_angular_ranks = {}
sensor_map_angular_center_ranks = {}
sensor_map_manhattan_ranks = {}
sensor_map_chebyshev_ranks = {}
sensor_map_window_ranks = {}
sensor_map_center_ranks = {}
sensor_map_hdist_ranks = {}
sensor_map_vdist_ranks = {}

min_euclidean=sys.float_info.max
max_euclidean=sys.float_info.min
min_manhattan=sys.float_info.max
max_manhattan=sys.float_info.min
min_chebyshev=sys.float_info.max
max_chebyshev=sys.float_info.min
min_hdist=sys.float_info.max
max_hdist=sys.float_info.min
min_vdist=sys.float_info.max
max_vdist=sys.float_info.min

pbar = tqdm(total=len(all_sensors))
for ind,sens in enumerate(all_sensors):
    
    euclidean_ranks = []
    angular_ranks = []
    angular_center_ranks = []
    manhattan_ranks = []
    chebyshev_ranks = []
    hdist_ranks = []
    vdist_ranks = []
    window_ranks = []
    center_ranks = []

    sens_x, sens_y = all_training_data.query("node == @sens").iloc[0][['coord_x', 'coord_y']].values
    other_sensors = sorted(list(set(all_sensors) - set([sens])))
    for other_sens in other_sensors: 
        other_sens_x, other_sens_y = all_training_data.query("node == @other_sens").iloc[0][['coord_x', 'coord_y']].values
        
        angular_distance = min(angle_between((sens_x, sens_y), (other_sens_x, other_sens_y)),
                                   angle_between((other_sens_x, other_sens_y), (sens_x, sens_y)))
        angular_center_distance = min(angle_between_center((sens_x, sens_y), (other_sens_x, other_sens_y)),
                                   angle_between_center((other_sens_x, other_sens_y), (sens_x, sens_y)))
        euclidean_distance = np.sqrt((sens_x - other_sens_x)**2 + (sens_y - other_sens_y)**2)
            
        
        manhattan_distance = abs(sens_x - other_sens_x) + abs(sens_y - other_sens_y)
        chebyshev_distance = max(abs(sens_x - other_sens_x), abs(sens_y - other_sens_y))

        hdist_distance = abs(sens_x - other_sens_x)
        vdist_distance = abs(sens_y - other_sens_y)
        
        #updates min max values 
        if euclidean_distance < min_euclidean: min_euclidean=euclidean_distance
        if euclidean_distance > max_euclidean: max_euclidean=euclidean_distance
        if manhattan_distance < min_manhattan: min_manhattan=manhattan_distance
        if manhattan_distance > max_manhattan: max_manhattan=manhattan_distance
        if chebyshev_distance < min_chebyshev: min_chebyshev=chebyshev_distance
        if chebyshev_distance > max_chebyshev: max_chebyshev=chebyshev_distance
        if hdist_distance < min_hdist: min_hdist=hdist_distance
        if hdist_distance > max_hdist: max_hdist=hdist_distance
        if vdist_distance < min_vdist: min_vdist=vdist_distance
        if vdist_distance > max_vdist: max_vdist=vdist_distance
            

        # the degree of similarity between two points, concerning their vicinity degree to a window
        sens_min_window_distance = min([np.sqrt((sens_x - w_x)**2 + (sens_y - w_y)**2) for (w_x, w_y) in window_coords])
        sens_other_min_window_distance = min([np.sqrt((other_sens_x - w_x)**2 + (other_sens_y - w_y)**2) for (w_x, w_y) in window_coords])
        window_distance_similarity = abs(sens_min_window_distance - sens_other_min_window_distance)
        
        # the degree of similarity between two points, concerning their vicinity to the center (5, 5)
        sens_center_distance = np.sqrt((sens_x - 5)**2 + (sens_y - 5)**2)
        sens_other_center_distance = np.sqrt((other_sens_x - 5)**2 + (other_sens_y - 5)**2)
        center_distance_similarity = abs(sens_center_distance - sens_other_center_distance)
        
        
        angular_ranks.append(angular_distance)
        angular_center_ranks.append(angular_center_distance)
        euclidean_ranks.append(euclidean_distance)
        manhattan_ranks.append(manhattan_distance)
        chebyshev_ranks.append(chebyshev_distance)
        hdist_ranks.append(hdist_distance)
        vdist_ranks.append(vdist_distance)
        window_ranks.append(window_distance_similarity)
        center_ranks.append(center_distance_similarity)

        
    sensor_map_angular_ranks[sens] = [(x,y) for y,x in sorted(zip(angular_ranks, other_sensors), reverse=False)]    
    sensor_map_angular_center_ranks[sens] = [(x,y) for y,x in sorted(zip(angular_center_ranks, other_sensors), reverse=False)]
    sensor_map_euclidean_ranks[sens] = [(x,y) for y,x in sorted(zip(euclidean_ranks, other_sensors), reverse=False)]
    sensor_map_manhattan_ranks[sens] = [(x,y) for y,x in sorted(zip(manhattan_ranks, other_sensors), reverse=False)]
    sensor_map_chebyshev_ranks[sens] = [(x,y) for y,x in sorted(zip(chebyshev_ranks, other_sensors), reverse=False)]  
    sensor_map_hdist_ranks[sens] = [(x,y) for y,x in sorted(zip(hdist_ranks, other_sensors), reverse=False)]  
    sensor_map_vdist_ranks[sens] = [(x,y) for y,x in sorted(zip(vdist_ranks, other_sensors), reverse=False)]
    sensor_map_window_ranks[sens] = [(x,y) for y,x in sorted(zip(window_ranks, other_sensors), reverse=False)]  
    sensor_map_center_ranks[sens] = [(x,y) for y,x in sorted(zip(center_ranks, other_sensors), reverse=False)]

    pbar.update(1)
pbar.close()

In [ ]:
# prepare datasets map for each sensors  

sensor_trainset = {} 
sensor_evalset = {} 
sensor_testset = {} 

sensor_cols = {}
perc=.8 #for genetic train/eval split

for sens in all_sensors:

    ## Preparing Training and Test Data
    
    X_train = all_training_data[all_training_data['node'] == sens]
    X_test = all_test_data[all_test_data['node'] == sens]
    
    np.random.seed(42)
    weeks_list = X_train.year_week.unique().tolist()
    np.random.shuffle(weeks_list)
    train_weeks = sorted(weeks_list[0:int(len(weeks_list)*perc)])

    X_eval = X_train[~ (X_train.year_week.isin(train_weeks))]
    X_train = X_train[X_train.year_week.isin(train_weeks)]
    
    # Temporal information + x y coords
    X_train = X_train[['moy_sin', 'moy_cos', 'dow_sin', 'dow_cos', 'seconds_from_midnight_sin', 'seconds_from_midnight_cos', 'coord_x', 'coord_y']]
    X_eval = X_eval[['moy_sin', 'moy_cos', 'dow_sin', 'dow_cos', 'seconds_from_midnight_sin', 'seconds_from_midnight_cos', 'coord_x', 'coord_y']]
    X_test = X_test[['moy_sin', 'moy_cos', 'dow_sin', 'dow_cos', 'seconds_from_midnight_sin', 'seconds_from_midnight_cos', 'coord_x', 'coord_y']] 

    
    
        
    # Reference temperatures
    for ref_sensor in all_sensors:
        X_train[ref_sensor+'_temperature'] = all_training_data[(all_training_data['node'] == ref_sensor) & (all_training_data.year_week.isin(train_weeks))]['temperature'].values
        X_eval[ref_sensor+'_temperature'] = all_training_data[(all_training_data['node'] == ref_sensor) & (~ all_training_data.year_week.isin(train_weeks))]['temperature'].values
        X_test[ref_sensor+'_temperature'] = all_test_data[all_test_data['node'] == ref_sensor]['temperature'].values

    
    # In Loving memory of our columns names
    sensor_cols[sens] = X_train.columns
    
    y_train = X_train[sens+'_temperature'].values
    y_eval = X_eval[sens+'_temperature'].values
    y_test = X_test[sens+'_temperature'].values
    
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train) #RIP dataframe => RIP columns names
    if X_eval.shape[0] > 0: 
        X_eval = scaler.transform(X_eval) #RIP dataframe => RIP columns names
    X_test = scaler.transform(X_test) #RIP dataframe => RIP columns names
    
    # restore y col values
    X_train[:, list(sensor_cols[sens]).index(sens+'_temperature')] = y_train
    if X_eval.shape[0] > 0: 
        X_eval[:, list(sensor_cols[sens]).index(sens+'_temperature')] = y_eval
    X_test[:, list(sensor_cols[sens]).index(sens+'_temperature')] = y_test    
    
    sensor_trainset[sens] = X_train
    sensor_evalset[sens] = X_eval
    sensor_testset[sens] = X_test

# END prepare datasets map for each sensors

In [ ]:
# Compute distance metric GENETIC Programming algorithm (New version with GP tree)

# Now let us turn to the genetic algorithm 

import random
random.seed(20885520)#572948150)#33087541)#355118678419#350145987#4296546)#42#572948150

from sklearn.linear_model import LinearRegression
from sklearn import metrics
from operator import attrgetter
import operator
import itertools

reset_maps = False

ranks = {
#          'angular':sensor_map_angular_ranks,
         'euclidean':sensor_map_euclidean_ranks,
         'manhattan':sensor_map_manhattan_ranks,
         'chebyshev':sensor_map_chebyshev_ranks
}

# Now define the genetic algorithm

if reset_maps:
    fitness_map={}
    predictors_results={}
                         
# Now we define the evaluation function for each individual
def evalIndividual(parameters):
    global fitness_map, predictors_results
    
    individual = parameters[0]
    func = toolbox.compile(expr=individual)
                         
    results_k_sens=np.array([0.]*(len(all_sensors)-1))
    
    
    for ind, sens in enumerate(all_sensors):
        # prepare the combined rank
        other_sens = sorted(set(all_sensors) - set([sens]))
        sensor_corrs = []
        for j in range(len(other_sens)):
            sensor_corrs+=[func(#sorted(ranks['angular'][sens])[j][1], 
                                (sorted(ranks['euclidean'][sens])[j][1]-min_euclidean)/(max_euclidean - min_euclidean),
                                (sorted(ranks['manhattan'][sens])[j][1]-min_manhattan)/(max_manhattan - min_manhattan),
                                (sorted(ranks['chebyshev'][sens])[j][1]-min_chebyshev)/(max_chebyshev - min_chebyshev))]
                         
#         print('Distance estimation', sensor_corrs)
        # sort the correlations to obtain a rank from worst (higher distance) to best
        sensor_corrs = zip(other_sens,sensor_corrs)
        sensor_corrs = sorted(sensor_corrs, key=lambda x: x[1], reverse=True)
        
        rank_key=','.join([x[0] for x in sensor_corrs])
        if rank_key in fitness_map:
            # fitness already computed for this ranking
            local_k_sens_results=fitness_map[rank_key]
        else:
            # we still have to compute the fitness for this sensors ranking
            local_k_sens_results=[]
            for k in range(0, len(all_sensors)-1): # from no discarded sensor to all but one discarded sensors

                ## preparing the dataset
                sensors_to_avoid = [x for (x, _) in sensor_corrs[0:k]]

                ## extrapolate trainset and testset from traindata and testdata                
                predictors_cols = [col for col in sensor_cols[sens] if (not sens in col) and not any(sensor_to_avoid in col for sensor_to_avoid in sensors_to_avoid)] 
#                 print('predictors_cols:', predictors_cols)

                predictors_key=sens+':'+','.join(sorted(predictors_cols))
    
                if predictors_key in predictors_results:
                    local_k_sens_results += [predictors_results[predictors_key]]
                else:
                    X_train = sensor_trainset[sens][:,  [list(sensor_cols[sens]).index(x) for x in predictors_cols ]]
                    X_eval = sensor_evalset[sens][:, [list(sensor_cols[sens]).index(x) for x in predictors_cols ]]
                    y_train = sensor_trainset[sens][:, list(sensor_cols[sens]).index(sens+'_temperature')]
                    y_eval = sensor_evalset[sens][:, list(sensor_cols[sens]).index(sens+'_temperature')]

#                     reg = LinearRegression(n_jobs=31).fit(X_train, y_train)
                    reg = XGBRegressor(n_jobs=31, random_state=42).fit(X_train, y_train)
                    predictions = reg.predict(X_eval)

                    errors = np.abs(y_eval - predictions)

                    ecdf_x = np.sort(errors)
                    ecdf_y = np.arange(1, len(ecdf_x) + 1) / len(ecdf_x)
                    perc_95 = int(len(ecdf_x)*0.95)
                    ecdf_area_95 = np.trapz(ecdf_y[:perc_95], ecdf_x[:perc_95])

                    local_k_sens_results += [ecdf_area_95]
                    predictors_results[predictors_key] = ecdf_area_95
            local_k_sens_results=np.array(local_k_sens_results)
            fitness_map[rank_key]=local_k_sens_results
        results_k_sens+=local_k_sens_results
    final_fitness = np.trapz(results_k_sens, list(range(0, len(results_k_sens))))
    
    return final_fitness,



# Now define the genetic operators
toolbox.register("evaluate", evalIndividual)

# toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("select", tools.selDoubleTournament, fitness_size=3, parsimony_size=1.4, fitness_first=True) # it encourages simple solutions
# NSGA-III requires a reference point set to guide the evolution into creating a uniform Pareto front in the objective space
#reference_points = tools.uniform_reference_points(nobj=1, p=12)  
#toolbox.register("select", tools.selNSGA3, ref_points=reference_points)
toolbox.register("mate", gp.cxOnePoint) # in-place operation
toolbox.register("mutate", gp.mutNodeReplacement, pset=pset) # in-place operation
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=15))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=15))


# Finally, we define the main function in which the genetic algorithm will run
def main_genetic_function(MAXGEN):
    list_best_fitnesses = []
    list_best_sols = []
    # Mutation and crossover probabilities, and maximum number of individuals
    CXPB, MUTPB = 0.7, 0.4 # cross 0.7, mut 0.4
    processes=31
    pop_size = processes*100 #10
    
    # Variable keeping track of the number of generation
    g = 0
    
    # Create the population
    pop = toolbox.population(n=pop_size)
        
    # Determine the validation fitnesses
    fitness_params = zip(pop, [g]*len(pop))
    #     fitnesses = list(map(toolbox.evaluate, fitness_params))
    
    # mp version
#     pool = multiprocessing.Pool(processes=processes)
#     fitnesses = pool.map(toolbox.evaluate, fitness_params)
#     pool.close()
#     pool.join()
    
    # sp version
    fitnesses = [toolbox.evaluate(par) for  par in fitness_params]
    
    # Determine the best fitness
    list_best_fitnesses.append(np.min(fitnesses))
                         
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    
    # Determine the best individual in the population
    best_sol_pop = min(pop, key=attrgetter("fitness"))
    list_best_sols.append(best_sol_pop)

    # Evolve the population
    fitnesses = [ind.fitness.values[0] for ind in pop]
    
    
    print("-- Generation %i --" % g)
    length = len(pop)
    mean = sum(fitnesses) / length
    sum2 = sum(x*x for x in fitnesses)
    std = abs(sum2 / length - mean**2)**0.5
    print("Fitnesses: Min %s" % round(min(fitnesses), 6), "  Max %s" % round(max(fitnesses), 6), "  Avg %s" % round(mean, 6), "  Std %s" % round(std, 6))
    print('Best individual', best_sol_pop)
    
    # Begin the evolution
    while g < MAXGEN:
        # A new generation
        g = g + 1
        print("-- Generation %i --" % g)
        # Select the next generation individuals, having the same size as the previous generation
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals: this ensures that we don’t use a reference to the individuals but a completely independent instance
        offspring = list(map(toolbox.clone, offspring))
         # Apply crossover and mutation on the offspring
        for i in range(1, len(offspring), 2): 
            if random.random() < CXPB:
                offspring[i-1], offspring[i] = toolbox.mate(offspring[i-1], offspring[i])
                del offspring[i-1].fitness.values
                del offspring[i].fitness.values
        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitness_params = zip(invalid_ind, [g]*len(invalid_ind))
                     
        # mp version
#         pool = multiprocessing.Pool(processes=processes)
#         fitnesses = pool.map(toolbox.evaluate, fitness_params)
#         pool.close()
#         pool.join()     
        
        # sp version
        fitnesses = [toolbox.evaluate(par) for par in fitness_params]
    
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        # Finally, replace the old population by the offspring, composed of some elements of the old population and some new elements
        pop[:] = offspring
        
        # Determine the fitness vals
        fitnesses= [ind.fitness.values[0] for ind in pop]
        list_best_fitnesses.append(np.min(fitnesses))

        # Determine best individual in the population 
        best_sol_pop = min(pop, key=attrgetter("fitness"))
        list_best_sols.append(best_sol_pop)
                         
        # Gather all the fitnesses in one list and print the stats (validation)
        length = len(pop)
        mean = sum(fitnesses) / length
        sum2 = sum(x*x for x in fitnesses)
        std = abs(sum2 / length - mean**2)**0.5
        print("Fitnesses: Min %s" % round(min(fitnesses), 6), "  Max %s" % round(max(fitnesses), 6), "  Avg %s" % round(mean, 6), "  Std %s" % round(std, 6))
        print('Best individual', best_sol_pop)
        write_pickle(zip(list_best_sols,list_best_fitnesses), 'genetic_partial_results.pickle')
    
    return list_best_sols, list_best_fitnesses


import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    list_top_solutions, list_top_fitnesses = main_genetic_function(300)

In [ ]:
# estimate rank map with genetic programming function

# Load best genetic programming function
# results = load_pickle('genetic_partial_results.pickle')
# gp_func=toolbox.compile(expr=list(results)[0][0])

def gp_func(ARG0,ARG1,ARG2):
    return my_max(ARG1, my_max(my_pow(my_square(ARG0), my_add(ARG1, ARG1)), my_add(ARG0, ARG2)))



ranks = {
#          'angular':sensor_map_angular_ranks,
         'euclidean':sensor_map_euclidean_ranks,
         'manhattan':sensor_map_manhattan_ranks,
         'chebyshev':sensor_map_chebyshev_ranks,
        }

sensor_map_old_weighted_ranks = {}
for ind, sens in enumerate(all_sensors):
    # prepare the combined rank
    other_sens = sorted(set(all_sensors) - set([sens]))
    sensor_corrs = []
    for j in range(len(other_sens)):
        sensor_corrs+=[gp_func(#sorted(ranks['angular'][sens])[j][1],
                       (sorted(ranks['euclidean'][sens])[j][1]-min_euclidean)/(max_euclidean - min_euclidean),
                       (sorted(ranks['manhattan'][sens])[j][1]-min_manhattan)/(max_manhattan - min_manhattan),
                       (sorted(ranks['chebyshev'][sens])[j][1]-min_chebyshev)/(max_chebyshev - min_chebyshev))]
        
    # sort the correlations to obtain a rank from best to worst (higher distance)
    sensor_corrs = zip(other_sens,sensor_corrs)
    sensor_map_old_weighted_ranks[sens] = sorted(sensor_corrs, key=lambda x: x[1], reverse=False)

In [ ]:
# estimating rank map

# So, for each sensor, I now determine the rank of the other sensors,
# based on the Kendall correlation value (from the most correlated, to the lowest)
# Using training data only

#!pip install saxpy
# https://cs.gmu.edu/~jessica/SAX_DAMI_preprint.pdf
# While it’s intuitive that larger alphabet sizes yield better results, there are diminishing returns
# as a increases. If space is an issue, an alphabet size in the range 5 to 8 seems to be a good choice
# that offers a reasonable balance between space and tightness of lower bound – each alphabet within
# this range can be represented with just 3 bits. Increasing the alphabet size would require more
# bits to represent each alphabet. 

    
    
from xgboost import XGBRegressor
import importlib

from saxpy.alphabet import cuts_for_asize
from saxpy.znorm import znorm
from saxpy.sax import ts_to_string

from sklearn.model_selection import train_test_split

import zlib
import sys
import shap

# from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

# Maps store the results from the best to the worst sensor, according to the specific metric
sensor_map_kendall_ranks = {}
sensor_map_pearson_ranks = {}
sensor_map_spearman_ranks = {}
sensor_map_euclidean_ranks = {}
sensor_map_SAXCBD_ranks = {}
sensor_map_SHAP_ranks = {}


use_eval = False

condition=True
if use_eval:
    condition=(all_training_data.year_week.isin(train_weeks))

pbar = tqdm(total=len(all_sensors))
for ind, sens in enumerate(all_sensors):
    
    sens_values = all_training_data[(all_training_data.node == sens) & condition]['temperature'].values
    
    sens_sax = ts_to_string(znorm(sens_values), cuts_for_asize(8))
    sens_sax_compr = zlib.compress(sens_sax.encode())
    sens_sax_compr_size = sys.getsizeof(sens_sax_compr)
    
    kendall_corrs = []
    pearson_corrs = []
    spearman_corrs = []
    euclidean_dists = []
    xgb_errors = []
    saxcbd_comprs = []
    
    other_sensors = sorted(list(set(all_sensors) - set([sens])))
    for other_sens in other_sensors: 
        other_ind = list(all_sensors).index(other_sens)
        other_sens_values = all_training_data[(all_training_data.node == other_sens) & condition]['temperature'].values
        
        other_sens_sax = ts_to_string(znorm(other_sens_values), cuts_for_asize(8))
        other_sens_sax_compr = zlib.compress(other_sens_sax.encode())
        other_sens_sax_compr_size = sys.getsizeof(other_sens_sax_compr)
        both_sens_sax_compr = zlib.compress((sens_sax + other_sens_sax).encode())
        both_sens_sax_compr_size = sys.getsizeof(both_sens_sax_compr)
        cbd_ratio = both_sens_sax_compr_size / (sens_sax_compr_size + other_sens_sax_compr_size) 
    
        kendall_corrs.append(1./(1.+abs(kendalltau(sens_values, other_sens_values)[0])))
        pearson_corrs.append(1./(1.+abs(np.corrcoef(sens_values, other_sens_values)[0, 1])))
        spearman_corrs.append(1./(1.+abs(spearmanr(sens_values, other_sens_values)[0])))
        euclidean_dists.append(np.linalg.norm(sens_values - other_sens_values))
        
        saxcbd_comprs.append(cbd_ratio)
    
    
    # Compute Shapley values
    train_xgb = pd.DataFrame()
    for other_sens in other_sensors:
        train_xgb[other_sens + '_temperature'] =  all_training_data[(all_training_data.node == other_sens) & condition]['temperature'].values

    reg = XGBRegressor(n_estimators=100, n_jobs=31, random_state=42).fit(train_xgb, sens_values)

    subsampled = shap.sample(train_xgb, 1000)
    explainer = shap.TreeExplainer(reg, subsampled)
    shap_values = explainer.shap_values(subsampled)
    average_shaps = 1./(1.+np.average(abs(shap_values), axis=0))
    feature_names = [x.split("_")[0] for x in train_xgb.columns]
    sensor_map_SHAP_ranks[sens] = [(x, y) for (y, x) in sorted(zip(average_shaps, feature_names), key=lambda pair: pair[0], reverse=False)]
        
    sensor_map_kendall_ranks[sens] = [(x,y) for y,x in sorted(zip(kendall_corrs, other_sensors), reverse=False)]
    sensor_map_pearson_ranks[sens] = [(x,y) for y,x in sorted(zip(pearson_corrs, other_sensors), reverse=False)]
    sensor_map_spearman_ranks[sens] = [(x,y) for y,x in sorted(zip(spearman_corrs, other_sensors), reverse=False)]
    sensor_map_euclidean_ranks[sens] = [(x,y) for y,x in sorted(zip(euclidean_dists, other_sensors), reverse=False)]
    sensor_map_SAXCBD_ranks[sens] = [(x,y) for y,x in sorted(zip(saxcbd_comprs, other_sensors), reverse=False)]
    
    pbar.update(1)
pbar.close()

write_pickle(sensor_map_kendall_ranks, 'ranks/sensor_map_kendall_ranks.pickle')
write_pickle(sensor_map_pearson_ranks, 'ranks/sensor_map_pearson_ranks.pickle')
write_pickle(sensor_map_spearman_ranks, 'ranks/sensor_map_spearman_ranks.pickle')
write_pickle(sensor_map_euclidean_ranks, 'ranks/sensor_map_euclidean_ranks.pickle')
write_pickle(sensor_map_SAXCBD_ranks, 'ranks/sensor_map_SAXCBD_ranks.pickle')
write_pickle(sensor_map_SHAP_ranks, 'ranks/sensor_map_SHAP_ranks.pickle')

In [ ]:
sensor_map_kendall_ranks = load_pickle('ranks/sensor_map_kendall_ranks.pickle')
sensor_map_pearson_ranks = load_pickle('ranks/sensor_map_pearson_ranks.pickle')
sensor_map_spearman_ranks = load_pickle('ranks/sensor_map_spearman_ranks.pickle')
sensor_map_euclidean_ranks = load_pickle('ranks/sensor_map_euclidean_ranks.pickle')
sensor_map_SAXCBD_ranks = load_pickle('ranks/sensor_map_SAXCBD_ranks.pickle')
sensor_map_SHAP_ranks = load_pickle('ranks/sensor_map_SHAP_ranks.pickle')

In [ ]:
# Testing  ranks on models preditions

from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression


results_k_sens = pd.DataFrame(columns=['rank', 'predicted_sensor', 'n_discarded_sensors', 'ecdf_area_95th'])


# must be metrics by which low values mean better values
ranks = {
#          'angular':sensor_map_angular_ranks,
#          'angular_center':sensor_map_angular_center_ranks,
         'euclidean':sensor_map_euclidean_ranks,
         'manhattan':sensor_map_manhattan_ranks,
         'chebyshev':sensor_map_chebyshev_ranks,
#          'window':sensor_map_window_ranks,
#          'center':sensor_map_center_ranks,
         'gp_function':sensor_map_weighted_ranks,
         'kendall': sensor_map_kendall_ranks,
         'pearson':sensor_map_pearson_ranks,
#          'spearman':sensor_map_spearman_ranks,  
         'shap': sensor_map_SHAP_ranks,
         'saxcbd': sensor_map_SAXCBD_ranks    
}


model = "XGBoostRegression"
# LinearRegression   XGBoostRegression  

pbar_outer = tqdm(total=len(ranks)*len(all_sensors))
for key in ranks:
    
    curmap = ranks[key]
    
    #pbar = tqdm(total=len(list(counts_per_sensor.index)))
    for ind, sens in enumerate(all_sensors):
        sensor_corrs = curmap[sens]

        sensor_corrs = sorted(sensor_corrs, key=lambda x: x[1], reverse=True) # in this way, we have the sensors from the worst to the best 

        #print(sensor_corrs)
        
        for k in range(0, len(all_sensors)-1): # from no discarded sensor to all but one discarded sensors
            
            ## preparing the dataset
            sensors_to_avoid = [x for (x, _) in sensor_corrs[0:k]]
            ref_sensors_here = sorted((set(all_sensors) - set(sensors_to_avoid)) - set([sens]))

            ## extrapolate trainset and testset from traindata and testdata
            predictors_cols = [col for col in sensor_cols[sens] if (not sens in col) and not any(sensor_to_avoid in col for sensor_to_avoid in sensors_to_avoid)] 
            X_train = sensor_trainset[sens][:,  [list(sensor_cols[sens]).index(x) for x in predictors_cols ]]
            X_test = sensor_testset[sens][:, [list(sensor_cols[sens]).index(x) for x in predictors_cols ]]
            y_train = sensor_trainset[sens][:, list(sensor_cols[sens]).index(sens+'_temperature')]
            y_test = sensor_testset[sens][:, list(sensor_cols[sens]).index(sens+'_temperature')]

            if model == 'LinearRegression':
                reg = LinearRegression(n_jobs=31).fit(X_train, y_train)
            elif model == 'XGBoostRegression':
                reg = XGBRegressor(n_estimators=100, n_jobs=2, random_state=42).fit(X_train, y_train)
            else:
                assert False
            
            predictions = reg.predict(X_test)

            errors = np.abs(y_test - predictions)

            ecdf_x = np.sort(errors)
            ecdf_y = np.arange(1, len(ecdf_x) + 1) / len(ecdf_x)
            perc_95 = int(len(ecdf_x)*0.95)
            ecdf_area_95 = np.trapz(ecdf_y[:perc_95], ecdf_x[:perc_95])

            new_row = pd.DataFrame()
            new_row['rank'] = [key]
            new_row['predicted_sensor'] = [sens]
            new_row['n_discarded_sensors'] = [k]
            new_row['ecdf_area_95th'] = [ecdf_area_95]
            results_k_sens = results_k_sens.append(new_row)

        pbar_outer.update(1)
pbar_outer.close()

write_pickle(results_k_sens, 'results_k_sens_' + model + '.pickle')

In [ ]:
ranks = { 
#          'angular':sensor_map_angular_ranks,
#          'angular_center':sensor_map_angular_center_ranks,
         'euclidean':sensor_map_euclidean_ranks,
         'manhattan':sensor_map_manhattan_ranks,
         'chebyshev':sensor_map_chebyshev_ranks,
#          'window':sensor_map_window_ranks,
#          'center':sensor_map_center_ranks,
         'gp_function':sensor_map_weighted_ranks,
         'kendall': sensor_map_kendall_ranks,
         'pearson':sensor_map_pearson_ranks,
#          'spearman':sensor_map_spearman_ranks,
         'shap': sensor_map_SHAP_ranks,
         'saxcbd': sensor_map_SAXCBD_ranks    
}

rank_labels={
     'euclidean': 'Euclidean',
     'manhattan': 'Manhattan',
     'chebyshev': 'Chebyshev',
     'gp_function': 'GP_function',
     'kendall': 'Kendall',
     'pearson': 'Pearson',
     'spearman' : 'Spearman',
     'shap': 'SHAP',
     'saxcbd': 'SAX-CBD'
}


model='XGBoostRegression'
# LinearRegression
# XGBoostRegression

results_k_sens = load_pickle('results_k_sens_' + model + '.pickle')

plt.figure(figsize=(12, 5))
# plt.rcParams["font.size"] = "11"
plot_string = "Area under the curve: \n"


ranks_errors_list = []


# get  list of methods ordered by error AUC
maxval = -1
for key in ranks:
    subresults = results_k_sens.query("rank == @key")
    grouped = subresults.groupby(['n_discarded_sensors']).sum()
    maxval = np.max(grouped['ecdf_area_95th']) if np.max(grouped['ecdf_area_95th']) > maxval else maxval
    
    auc = round(np.trapz(grouped['ecdf_area_95th'], list(range(0, len(grouped['ecdf_area_95th'])))), 2)
    ranks_errors_list += [ (auc,key)]

ranks_errors_list = sorted(ranks_errors_list, reverse=False)


maxval = -1
for key in [ k for (_,k) in ranks_errors_list ]:
    subresults = results_k_sens.query("rank == @key")
    grouped = subresults.groupby(['n_discarded_sensors']).sum()
    maxval = np.max(grouped['ecdf_area_95th']) if np.max(grouped['ecdf_area_95th']) > maxval else maxval
    plt.plot(range(1,len(grouped['ecdf_area_95th'])+1), grouped['ecdf_area_95th'].tolist()[::-1], label=rank_labels[key])
    
    plot_string += "  > " + rank_labels[key] + ": " + str(round(np.trapz(grouped['ecdf_area_95th'], list(range(0, len(grouped['ecdf_area_95th'])))), 2)) + "\n"
plt.xlabel("Number of selected sensors (from best to worst according to the rank)")
# plt.xlim(1,11)
plt.ylabel("Sum of 95th percentile error (temperature)")
# plt.title("Performance of " + model + ", discarding sensors based on training set ranks")
plt.legend(title="Ranking strategy")
plt.text(x=7.2, y=maxval-1.79, s=plot_string)
plt.savefig('plots/Selected_sensor_results_k_sens_ProxyMetrics_' + model + '.pdf', bbox_inches='tight', pad_inches=0, dpi=1000)
plt.show()

In [ ]:
# For all the pearson ranks we estimate how many times a sensor is present in the first n_refs = 4 (elbow) positions

results_best_sens = {}
for s in all_sensors:
    results_best_sens[s]=0

n_refs = 11 

# must be metrics by which low values mean better values
ranks = {
         'angular':sensor_map_angular_ranks,
         'euclidean':sensor_map_euclidean_ranks,
         'manhattan':sensor_map_manhattan_ranks,
         'chebyshev':sensor_map_chebyshev_ranks,
         'gp_function':sensor_map_weighted_ranks,
         'pearson':sensor_map_pearson_ranks,
         'shap': sensor_map_SHAP_ranks,
         'saxcbd': sensor_map_SAXCBD_ranks    
}

distance='pearson'

rank = ranks[distance]

epsilon=0.1
sensors_error = results_k_sens.query('rank==@distance').groupby('predicted_sensor')[['ecdf_area_95th']].sum().values
sensors_weight = np.subtract(1, np.divide(sensors_error, np.max(sensors_error+epsilon)))

for i,sens in enumerate(all_sensors):
    sensors_rank = rank[sens]
    sensors_rank = sorted(sensors_rank, key=lambda x: x[1], reverse=False) # in this way, we have the sensors from the best to the worst

    sensors_rank = [x for (x, _) in sensors_rank[0:n_refs]]

     # first j sensors in the rank
    for j in range(n_refs):
        results_best_sens[sensors_rank[j]]+=(n_refs-j)*sensors_weight[i][0]
        
# print(results_best_sens)

plt.figure(figsize=(14,4))

results_best_sens=sorted([x for x in zip(results_best_sens.keys(), results_best_sens.values())], key=lambda pair: pair[1], reverse=True)

keys=[x[0] for x in results_best_sens]
values=[x[1] for x in results_best_sens]


kn = KneeLocator(range(len(values)), values, curve='convex', direction='decreasing')
print('elbow:', kn.knee+1)

plt.bar(range(len(values)), height=values)
plt.plot(range(len(values)), values, color='b')

plt.xticks(range(len(keys)), keys, rotation=35, ha="right",rotation_mode="anchor")
plt.ylabel('Weighted Borda count vote')# 'Borda count vote considering '+str(n_refs)+' positions'

plt.savefig('plots/best_ranked_sensors_'+ distance +'.pdf', dpi=600, bbox_inches='tight', pad_inches=0)
plt.show()